In [1]:
import itertools
import functools

import pandas as pd
import slr.psmsl
import slr.wind
import slr.models
import slr.tables

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [2]:
default_dataset_name = 'rlr_annual'

stations = ['NL', 'NL-Delfzijl', 'Vlissingen', 'Hoek van Holland', 'Den Helder', 'Delfzijl', 'Harlingen', 'IJmuiden']
wind_variants = ['NCEP1', '20CR', 'Combined']
model_variants = ['linear', 'quadratic', 'broken_linear', 'broken_quadratic']
quantities = ['height', 'height - surge anomaly']
reference_point_wind = {"lat": 53, "lon": 3}


In [3]:
work_df = pd.DataFrame(
    list(
        itertools.product(
            stations, wind_variants, model_variants
        )
    ), 
    columns=['station', 'wind', 'model']
)
work_df

,station,wind,model
0,NL,NCEP1,linear
1,NL,NCEP1,quadratic
2,NL,NCEP1,broken_linear
3,NL,NCEP1,broken_quadratic
4,NL,20CR,linear
...,...,...,...
91,IJmuiden,20CR,broken_quadratic
92,IJmuiden,Combined,linear
93,IJmuiden,Combined,quadratic
94,IJmuiden,Combined,broken_linear


# Stations
Let's first collect all the info of all the stations. We have the main stations and aggregations that are combinations of multiple stations.

In [4]:
# get the zipfiles
zipfiles = slr.psmsl.get_zipfiles()
# get the list of main stations
main_stations = slr.psmsl.get_main_stations()

# get our zipfile
zipfile = zipfiles[default_dataset_name]
# load the stations
stations = slr.psmsl.get_station_list(zipfiles[default_dataset_name], dataset_name=default_dataset_name)

# the dutch stations in the PSMSL database, make a copy
# or use stations.coastline_code == 150 for all dutch stations
selected_stations = pd.merge(main_stations, stations, left_index=True, right_index=True, suffixes=['_rws', '_psmsl'])

# these are the individual stations
selected_stations

,name_rws,msl-rlr,msl-nap,nap-rlr,alpha,ddl_id,location,psmsl_id,foundation_low,station_low,...,links,lat,lon,name_psmsl,coastline_code,station_code,quality,met_monthly_url,rlr_monthly_url,rlr_annual_url
id,,,,,,,,,,,,,,,,,,,,,
20,Vlissingen,6976,46,6930,118,VLISSGN,Vlissingen,20,-17.6,2.50,...,[{'href': 'http://gnss1.tudelft.nl/dpga/statio...,51.442222,3.596111,VLISSINGEN,150,101,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...
22,Hoek van Holland,6987,114,6873,117,HOEKVHLD,Hoek van Holland,22,-3.3,5.27,...,[{'href': 'http://geodesy.unr.edu/NGLStationPa...,51.977500,4.120000,HOEK VAN HOLLAND,150,51,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...
23,Den Helder,6962,16,6946,107,DENHDR,Den Helder,23,-5.0,5.00,...,[],52.964444,4.745000,DEN HELDER,150,31,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...
24,Delfzijl,6953,130,6823,117,DELFZL,Delfzijl,24,-20.0,1.85,...,[{'href': 'http://geodesy.unr.edu/NGLStationPa...,53.326389,6.933056,DELFZIJL,150,1,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...
25,Harlingen,7024,110,6914,108,HARLGN,Harlingen,25,-5.4,5.55,...,[],53.175556,5.409444,HARLINGEN,150,21,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...
32,IJmuiden,7014,64,6950,110,IJMDBTHVN,IJmuiden,32,-13.0,4.20,...,[{'href': 'http://gnss1.tudelft.nl/dpga/statio...,52.462222,4.554722,IJMUIDEN,150,41,N,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...,http://www.psmsl.org/data/obtaining/rlr.diagra...


# Wind x Stations
Now we can create a dataset with measurements per  (station, wind)  combination. 

In [5]:
stations_with_wind = []
for wind_variant in wind_variants:
    stations = slr.psmsl.add_series_to_stations(selected_stations.copy(), wind_product=wind_variant)
    # aggregate stations (NL + NL - Delfzijl)
    stations = slr.psmsl.add_aggregated_stations(stations)
    stations['wind_variant'] = wind_variant

    stations_with_wind.append(stations)
stations_with_wind_df = pd.concat(stations_with_wind)


found point 52.3799 3.75
found point 53.0 3.0
found point 52.3799 3.75
found point 53.0 3.0
found point 52.3799 3.75
found point 53.0 3.0


# Wind x stations x models



In [6]:
results = []
models = {
    "linear": slr.models.linear_model,
    "broken_linear": slr.models.broken_linear_model,
    "broken_quadratic": slr.models.broken_quadratic_model,
    "quadratic": slr.models.quadratic_model,
}

fits = []
for psmsl_id, station in stations_with_wind_df.iterrows():
    df = station[default_dataset_name]
    for model_variant in model_variants:
        model = models[model_variant]
        for quantity in quantities:
            if 'surge' in quantity:
                with_wind = False
            else:
                with_wind = True
            fit, names = model(df, with_wind=with_wind, quantity=quantity)
            fits.append({
                "name": station["name"],
                "fit": fit,
                "names": names,
                "wind_variant": station["wind_variant"],
                "model_variant": model_variant,
                "quantity": quantity
            })
        
 
    

In [7]:
slr.tables.fits_to_fits_df(fits)

,name,fit,names,wind_variant,model_variant,quantity,aic,F,df_model,prediction,has_tide,has_wind,prediction_mean_tide,prediction_mean_wind,prediction_mean_tide_mean_wind,exog
0,NaN,<statsmodels.regression.linear_model.Regressio...,"[Constant, Trend, Nodal U, Nodal V, Wind $u^2$...",NCEP1,linear,height,1518.820112,22.360908,5.0,<statsmodels.regression._prediction.Prediction...,True,True,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,Constant Trend Nodal U Nodal V Wind...
1,NaN,<statsmodels.regression.linear_model.Regressio...,"[Constant, Trend, Nodal U, Nodal V]",NCEP1,linear,height - surge anomaly,1522.065655,14.027821,3.0,<statsmodels.regression._prediction.Prediction...,True,False,<statsmodels.regression._prediction.Prediction...,NaN,NaN,Constant Trend Nodal U Nodal V 0 ...
2,NaN,<statsmodels.regression.linear_model.Regressio...,"[Constant (in year 1970), Trend, Acceleration,...",NCEP1,quadratic,height,1515.526296,24.697489,6.0,<statsmodels.regression._prediction.Prediction...,True,True,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,Constant (in year 1970) Trend Accelerat...
3,NaN,<statsmodels.regression.linear_model.Regressio...,"[Constant (in year 1970), Trend, Acceleration,...",NCEP1,quadratic,height - surge anomaly,1517.451104,38.171808,4.0,<statsmodels.regression._prediction.Prediction...,True,False,<statsmodels.regression._prediction.Prediction...,NaN,NaN,Constant (in year 1970) Trend Accelerat...
4,NaN,<statsmodels.regression.linear_model.Regressio...,"[Constant, Trend, +trend (1993), Nodal U, Noda...",NCEP1,broken_linear,height,1519.196481,18.508232,6.0,<statsmodels.regression._prediction.Prediction...,True,True,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,Constant Trend +trend (1993) Nodal U ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,Netherlands (without Delfzijl),<statsmodels.regression.linear_model.Regressio...,"[Constant (in year 1970), Trend, Acceleration,...",Combined,quadratic,height - surge anomaly,1205.251503,307.105718,4.0,<statsmodels.regression._prediction.Prediction...,True,False,<statsmodels.regression._prediction.Prediction...,NaN,NaN,Constant (in year 1970) Trend Accelerat...
188,Netherlands (without Delfzijl),<statsmodels.regression.linear_model.Regressio...,"[Constant, Trend, +trend (1993), Nodal U, Noda...",Combined,broken_linear,height,1147.798541,306.270948,6.0,<statsmodels.regression._prediction.Prediction...,True,True,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,Constant Trend +trend (1993) Nodal U ...
189,Netherlands (without Delfzijl),<statsmodels.regression.linear_model.Regressio...,"[Constant, Trend, +trend (1993), Nodal U, Noda...",Combined,broken_linear,height - surge anomaly,1202.302491,412.394681,4.0,<statsmodels.regression._prediction.Prediction...,True,False,<statsmodels.regression._prediction.Prediction...,NaN,NaN,Constant Trend +trend (1993) Nodal U ...
190,Netherlands (without Delfzijl),<statsmodels.regression.linear_model.Regressio...,"[Constant (in year 1960), Trend, Acceleration ...",Combined,broken_quadratic,height,1150.619468,294.684710,6.0,<statsmodels.regression._prediction.Prediction...,True,True,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,<statsmodels.regression._prediction.Prediction...,Constant (in year 1960) Trend Accelerat...
